In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_scoreimport pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [ ]:
nome_file='Friday-02-03-2018_TrafficForML_CICFlowMeter'

with open('pickle/'+nome_file+'/scaled_document.pickle', 'rb') as handle:

    df = pickle.load(handle)

In [ ]:
feature_1 = 'Init Bwd Win Byts'
feature_2 = 'Fwd Pkts/s'


df = df[[feature_1, feature_2,'Label']]

In [ ]:
df_prova = df.groupby(['Label'])['Label'].count()


df_prova=df_prova.to_frame()
print(df_prova)
df_prova.set_index('Label')
df_prova=df_prova.rename(columns={'Label':'Count'})

In [ ]:
bot = df[df.Label == 1]
benign = df[df.Label == 0]

print("Benign: ", len(benign), "Bot: ", len(bot))

In [ ]:
#bilanciamento del dataset
balanced_d = pd.concat([bot, benign.sample(len(bot))])
x = balanced_d.iloc[:,:-1]
y = balanced_d.iloc[:,-1:]
balanced_d.shape

In [ ]:
# Eseguiamo il clustering

n_clusters = len(balanced_d['Label'].unique())  # numero di cluster

y_pred = KMeans(n_clusters=n_clusters).fit_predict(x)

# Associazione del cluster i-esimo con la classe i-esima
cluster_class_mapping = {}
for i in range(n_clusters):
    cluster_samples = x[y_pred == i]
    cluster_classes = y[y_pred == i]
    unique_classes, class_counts = np.unique(cluster_classes, return_counts=True)
    dominant_class = unique_classes[np.argmax(class_counts)]
    cluster_class_mapping[i] = dominant_class

In [ ]:
difference = pd.DataFrame(data=[],columns=['num_cluster','num_class'])

In [ ]:
# Plot dei risultati
plt.scatter(x[feature_1], x[feature_2], c=y['Label'])
plt.title("Optimal Number of Clusters")
plt.show()

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.scatter(x[feature_1], x[feature_2], c=y_pred)
axs.set_title("Mixture of Gaussian Blobs")

# Stampiamo l'associazione del cluster con la classe
for i in range(n_clusters):
    if(i != cluster_class_mapping[i]):

        new_row = pd.Series({'num_cluster': i, 'num_class': cluster_class_mapping[i]})
        difference = pd.concat([difference,new_row.to_frame().T],ignore_index=True)
        
    axs.text(
        np.mean(x[feature_1][y_pred == i]), np.mean(x[feature_2][y_pred == i]),
        f"Cluster {i}: Class {cluster_class_mapping[i]}",
        fontsize=12, fontweight='bold', color='red', ha='center', va='center'
    )

plt.suptitle("Ground truth clusters").set_y(0.95)
plt.show()

In [ ]:
indici = []

for a in difference['num_cluster']:
    indici.append([
    index for index in range(len(y_pred))
    if y_pred[index] == a
])

for ind,true_value in zip(indici,difference['num_class']):
    for a in ind:
        y_pred[a] = true_value

In [ ]:
print(accuracy_score(y_pred,y))

In [ ]:
bal_x = balanced_d.iloc[:,:-1]

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random

prova = bal_x.values
neigh = NearestNeighbors(n_neighbors=5, n_jobs=4, algorithm='ball_tree', leaf_size=5)
nbrs = neigh.fit(prova)
distances, indices = nbrs.kneighbors(prova)

distances = np.sort(distances, axis=0)
plt.figure(figsize=(12,8))
plt.plot(distances[:,1])

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=1, min_samples=6).fit(prova)
ymeans = db.labels_
ymeans

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs
from minisom import MiniSom

# Definisci le dimensioni della mappa
map_width = 10
map_height = 10

# Inizializza la SOM
som = MiniSom(map_width, map_height, 2, sigma=1.0, learning_rate=0.5)

# Inizializza i pesi in modo casuale
som.random_weights_init(bal_x)

# Addestra la SOM
som.train_batch(bal_x, 1000)

# Trova il BMU (Best Matching Unit) per ogni campione di input
bmu_indexes = np.array([som.winner(x) for x in bal_x])

# Stampa i BMU e i relativi vettori di peso
for i, (x, bmu) in enumerate(zip(bal_x, bmu_indexes)):
    print(f"Campione {i+1}:")
    print("Input:", x)
    print("BMU Indice:", bmu)
    print("BMU Peso:", som.weights[bmu[0], bmu[1]])
    print()